In [1]:
import csv
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import sqlite3
from pprint import pprint as pp

In [ ]:
def connect_SQLiteDB():
    try:
        dbconn = sqlite3.connect('SQLite_District.db')
        cursor = dbconn.cursor()
        print()
        print("Successfully Connected to SQLite Database")
               
    except sqlite3.Error as error:
        print("Error while connecting to SQLite", error)
        dbconn = handle_DB_Error(dbconn, cursor)
        
    return dbconn, cursor

In [ ]:
def create_Table(dbconn, cursor):

    query_create_table ='''CREATE TABLE IF NOT EXISTS
                            districts(
                            district_id INTEGER PRIMARY KEY, 
                            State_abbrev TEXT,
                            black_population FLOAT, 
                            native_american_population FLOAT,
                            pacific_islander_population FLOAT,
                            email TEXT unique, 
                            password TEXT)'''

    
    
    
    percentage_df['black_%'] = return_df['black'] / return_df['total'] * 100
    percentage_df['na_%'] = return_df['native amer'] / return_df['total'] * 100
    percentage_df['pi_%'] = return_df['pacific is'] / return_df['total'] * 100
    percentage_df['other_%'] = return_df['other'] / return_df['total'] * 100
    percentage_df['white_%'] = return_df['other'] / return_df['total'] * 100
    percentage_df['hispanic_%'] = return_df['hispanic'] / return_df['total'] * 100 
# Type Conversions:
# Python Type   SQLite Type
# -----------   -----------
# str           TEXT
# int           INTEGER
# float         REAL
# byte          BLOB
# None          Null

    try:
        #Execute the query
        cursor.execute(query_create_table)
        #Commit the transaction
        dbconn.commit()
        print()
        print('Table created or accessed successfully.')
    
    except sqlite3.Error as error: 
        print('Error occurred when creating table.', error)
        dbconn = handle_DB_Error(dbconn, cursor)        
    
    #This will be None if there was an error
    return dbconn  

In [ ]:
def write_sql():
'''
    conn = sqlite3.connect('test_database')
    c = conn.cursor()

    c.execute('CREATE TABLE IF NOT EXISTS products (product_name text, price number)')
    conn.commit()

data = {'product_name': ['Computer','Tablet','Monitor','Printer'],
        'price': [900,300,450,150]
        }


    df.to_sql('products', conn, if_exists='replace', index = False)
 
   #c.execute('''  
    #SELECT * FROM products
              ''')

    for row in c.fetchall():
        print (row)
'''        

In [ ]:
def percentages_df(percentage_df):
    '''
    updates with rounded percentages
    Returns pandas dataframe    
    '''
    percentage_df['black_%'] = return_df['black'] / return_df['total'] * 100
    percentage_df['na_%'] = return_df['native amer'] / return_df['total'] * 100
    percentage_df['pi_%'] = return_df['pacific is'] / return_df['total'] * 100
    percentage_df['other_%'] = return_df['other'] / return_df['total'] * 100
    percentage_df['white_%'] = return_df['other'] / return_df['total'] * 100
    percentage_df['hispanic_%'] = return_df['hispanic'] / return_df['total'] * 100     
    
    #print(return_df) 
    
    rounded_df = percentage_df.round(2)
    
    print(rounded_df)
    
    return rounded_df

In [ ]:
def build_df(bar_lists):
    '''
    create pandas dataframe from list of lists
    Returns pandas dataframe    
    '''
    len_b = len(bar_lists)
    print('len_b',len_b)
    
    column_names = []
    row_values = []
    for b in range(len_b):
        row_b =[]
        b_item = bar_lists[b]
        #print('b_item',b_item)
        len_c = len(b_item)
        #print('len_c', len_c)
        for c in range(len_c):
            c_item = b_item[c]
            #print('c_item', c_item)
            if b == 0:
                column_names.append(c_item[0])    
            row_b.append(c_item[1])
        row_values.append(row_b)
    
    print('column_names',column_names)
    print('row_values',row_values)
    #return_df = []
    return_df = pd.DataFrame(row_values, columns = column_names)
    
    return return_df

In [2]:
def check_list(input_district_x,selected_districts):
    '''
    Determines if selected district number in not in the list
    Returns boolean if the selected district is good to add to list    
    '''
    #print('input_district_x', type(input_district_x), input_district_x)
    #print('selected_districts',selected_districts)
    valid_district = False
    if len(selected_districts) == 0:
        valid_district = True
    else:
        for x in range(len(selected_districts)):
            #print('x',x)
            if input_district_x == selected_districts[x]:
                valid_district = False 
            else:
                valid_district = True
    #print('valid_district', valid_district)    
    return(valid_district)

In [3]:
def select_districts(district_values):
    '''
    Prompts for district(s) to be selected
    Returns list of selected districts
    
    '''
    selected_districts = []
    numx = 5
    lenx = len(district_values)
    if lenx < numx:
        numx = lenx
    not_validiated = True
    
    print(f'Select up to {numx} of districts.  Enter zero to stop selecting')
    print(f'Valid districts numbers: 1 thru {lenx} (inclusive)')
    while not_validiated:
        input_district = input('Select ditrict: \n')
        if input_district.isdigit():
            input_district_x = int(input_district)
            if input_district_x == 0:
                not_validiated = False
            else:    
                if input_district_x > 0 and input_district_x <= lenx:
                    good_input = check_list(input_district_x,selected_districts)
                    if good_input:
                        selected_districts.append(input_district_x)
                        if len(selected_districts) == numx:
                            not_validiated = False
                        else:
                            continue
                    else:
                        print('Already in list, please try again')
                else:
                    print('Not valid input, please try again')
        else:
            print('Not valid input, please try again')
            
                
        
    #print('selected_districts',selected_districts)     
                        
    return(selected_districts)

In [4]:
def lists_from_dictionary_2(values,selected_districts):
    '''
    creates lists for the selected districts from each dictionary entry and calcualtes percentages
    
    '''   
    ret_list1 = []
    variables_names = ['DP05_0033E','DP05_0038E','DP05_0039E','DP05_0044E','DP05_0052E','DP05_0057E','DP05_0037E', 'DP05_0071E']
    pop_variables_names = ['total','black','native amer', 'asian', 'pacific is', 'other','white', 'hispanic']
    zipped = zip(variables_names,pop_variables_names)
    lookup_list = list(zipped)
    len3 = len(lookup_list)
    #print('lookup_list',lookup_list)   
           
    #loop thru selected districts 

    for dist_x in selected_districts:
        sub_list = []
        #print('dist_x=',dist_x)      
            
        #index_x = list_x.index(x)
        #print('index_x',index_x)
        #len1 = len(list_x)
        #if the list returned is one of the selected districts, add it to output 
        for x, row in enumerate(values):
        #for y in range(len1): 
            list_x = values[x]
            len_x = len(list_x)
            #print('list_x',x,list_x)
            district_num = list_x[1][1]
            #print('district_num', district_num)
            if dist_x == district_num: 
                for y in range(len_x):
                    y_item = list_x[y]
                    #print('y_item',y,y_item)
                    
                #change census variables to race names
                    if y > 1:
                        #print('y_item[0]',y_item[0])
                        for z in range(len3):
                            z_item = lookup_list[z]
                            #print('z_item',z,z_item)
                            if y_item[0] == z_item[0]:
                                y_item[0] = z_item[1]
                                #print('replaced',y_item)
                ret_list1.append(list_x)
                #print('ret_list1',ret_list1)    
        #ret_list1.append(sub_list)    
        #print('ret_list1',ret_list1)
    
    return  ret_list1 

In [ ]:
def build_census_dict_st(json_file, state_abbr):
    '''
    creates dictionary from census state data 
    state data will be district 0 
    return dictionary 
    '''
    census_dict = {}
    
    census_data = json.loads(json_file)
       
    names_list = []
    for x, row in enumerate(census_data):
        #print('row_data',row)
        row_len = len(row)
        census_list = [['state',state_abbr]]
        
        for y, column in enumerate(row):
            # use district 0 as dictionary key  
            if x == 0:
                #save population variable names
                if y <(row_len-2):
                    names_list.append(row[y]) 
            if x > 0:
                #print('district_words', district_words)
                census_dict_key = 0
                #print('key',census_dict_key)
                # build list of [variable_name,population_value], ignore last two columns 
                if y == 1:
                    census_list.append(['district',census_dict_key])
                if y > 0 and y <(row_len-2):
                    census_list.append([names_list[y],int(row[y])])
        if x > 0:            
            #print('census_list',census_list)        
            census_dict[census_dict_key] = census_list   
    
    
    return census_dict 

In [5]:
def lists_from_dictionary(values):
    '''
    creates lists from each dictionary entry and calcualtes percentages
    
    '''
    list1 = values[0]
    list2 = values[1]
    len1 = len(list1)
    len2 = len(list2)
    pop_var = []
    ret_list1 = []
    ret_list2 = []
    ret_list1h =[]
    ret_list2h = []
    print('values1',list1)
    print('len1',len1)
    print('len2',len2)
    print('values2',list2)
    
    #do not include total in list
    #for x in range(1,len1):
    for x in range(len1):    
        item = list1[x]
        #print('list1_item',x,item)
        #print('item[0]',item[0])
        pop_var.append(item[0])
        ret_list1.append(item[1]) 
        #print('pop_var',pop_var)
        
    #do not include total in list    
    for x in range(len2):
        #print('x2',x)
        item = list2[x]
        ret_list2.append(item[1]) 
    
    #these are the hispanic population, used in a separate graph since hispanic is not a race
    ret_list1h = ret_list1.pop(-1)  
    ret_list2h = ret_list2.pop(-1) 
    len1 = len(ret_list1)
    len2 = len(ret_list2)
    #print('len1',len1)
    #print('len2',len2)
    #print('ret_list1',ret_list1)
    #print('ret_list2',ret_list2)
    #print('ret_list1h',ret_list1h)
    #print('ret_list2h',ret_list2h)
    
    per_list1 = [] 
    #first item in the list is the total population for the county
    total1 = ret_list1[0]
    total1h = ret_list1h
    #these are the hispanic population, used in a separate graph since hispanic is not a race
    per_list1h = [round(((ret_list1h / total1)*100), 2)]
    
    tot_pop1 = sum(ret_list1) - total1         
    #print('total1', total1)
    #print('total1h', total1h)
    #print('tot_pop1',tot_pop1)
    #start looping after total
    for x in range(1,len1):
        #print('x',x)
        item = ret_list1[x]
        #print('item',x,item)
        idx = ret_list1.index(item)
        #print('idx', idx)
        percent = round(((item /tot_pop1)*100), 2)
        per_list1.append(percent)    
    
    per_list2 = []  
    #first item in the list is the total population for the county
    total2 = ret_list2[0]
    total2h = ret_list2h
    #these are the hispanic population, used in a separate graph since hispanic is not a race      
    per_list2h = [round(((ret_list2h / total2)*100), 2)]
    
    #print('total2', total2)
    #print('total2h', total2h)
    tot_pop2 = (sum(ret_list2)) - total2 
    #start looping after total
    for x in range(1,len2):
        item = ret_list2[x]
        #print('item',x,item)
        percent = round(((item /tot_pop2)*100), 2)
        per_list2.append(percent)
      
    #print('per_list1',per_list1)
    #print('per_list2',per_list2)
    #print('ret_list2',ret_list2)
    return pop_var, per_list1, per_list2, per_list1h, per_list2h

In [6]:
def build_census_dict(json_file, state_abbr):
    '''
    creates dictionary from census district data 
    return dictionary 
    '''
    census_dict = {}
    
    census_data = json.loads(json_file)
       
    names_list = []
    for x, row in enumerate(census_data):
        #print('row_data',row)
        row_len = len(row)
        census_list = [['state',state_abbr]]
        
        for y, column in enumerate(row):
            # use district number as dictionary key  
            if x == 0:
                #save population variable names
                if y <(row_len-2):
                    names_list.append(row[y]) 
            if x > 0:
                #print('names_list',names_list)
                #use first word as key
                district_words = row[0].split()
                #print('district_words', district_words)
                census_dict_key = int(district_words[2])
                #print('key',census_dict_key)
                # build list of [variable_name,population_value], ignore last two columns 
                if y == 1:
                    census_list.append(['district',census_dict_key])
                if y > 0 and y <(row_len-2):
                    census_list.append([names_list[y],int(row[y])])
        if x > 0:            
            #print('census_list',census_list)        
            census_dict[census_dict_key] = census_list   
    
    ''' 
    #create values for graphs
    legend_names = list(census_dict.keys())
    legend_names.sort()
    
    #print('legends',legend_names)
    #print('census_dict ')
    #pp(census_dict)
    values = list(census_dict.values())
    #print('values', len(values), values)
    
    
    #prompt for which districts to be selected
    selected_districts = select_districts(legend_names)
    #print('selected_districts',selected_districts) 
    
    #build a list of lists based on the selected disticts
    #pop_variables, bar1_list, bar2_list, bar1h_list, bar2h_list = lists_from_dictionary(values)
    #create list of lists
    bar_lists = lists_from_dictionary_2(values,selected_districts)
    
    ###write bar_lists to DB
    
    len_b = len(bar_lists)
    print('len_b',len_b)
    
    column_names = []
    row_values = []
    for b in range(len_b):
        row_b =[]
        b_item = bar_lists[b]
        #print('b_item',b_item)
        len_c = len(b_item)
        #print('len_c', len_c)
        for c in range(len_c):
            c_item = b_item[c]
            #print('c_item', c_item)
            if b == 0:
                column_names.append(c_item[0])    
            row_b.append(c_item[1])
        row_values.append(row_b)
    
    print('column_names',column_names)
    print('row_values',row_values)
    #return_df = []
    return_df = pd.DataFrame(row_values, columns = column_names)
    
    #write pandas to SQL
    
    
    
    #print('bar_lists',bar_lists)
    print(return_df)
    #print('bar2_list',bar2_list)
    
    return_df['black_%'] = return_df['black'] / return_df['total'] * 100
    return_df['na_%'] = return_df['native amer'] / return_df['total'] * 100
    return_df['pi_%'] = return_df['pacific is'] / return_df['total'] * 100
    return_df['other_%'] = return_df['other'] / return_df['total'] * 100
    return_df['white_%'] = return_df['other'] / return_df['total'] * 100
    return_df['hispanic_%'] = return_df['hispanic'] / return_df['total'] * 100     
    
    print(return_df) 
    
    new_df = return_df.round(2)
    
    print(new_df) 
       
    variables_names = ['DP05_0038E','DP05_0039E','DP05_0044E','DP05_0052E','DP05_0057E','DP05_0037E', 'DP05_0071E']
    pop_variables_names = ['black','native amer', 'asian', 'pacific is', 'other','white', 'hispanic']
    zipped = zip(variables_names,pop_variables_names)
    lookup_list = list(zipped)
    #print('lookup_list',lookup_list)   
    
    #lookup population name from lookup_list
    len3 = len(lookup_list)
    names = []
    #do not include total in list of names
    for x in range(len3):
        #print('x3',x)
        item = lookup_list[x]
        #print('item',item)
        if item[1] != 'total':
            names.append(item[1])
   
    namesh = [names.pop(-1)]
    y_pos = np.arange(len(bar1_list))
    #print('y_pos',y_pos)
    #print('names',names)
    #print('namesh',namesh)
    #print('bar1_list',bar1_list)
    #print('bar2_list',bar2_list)
    #print('bar1h_list',bar1h_list)
    #print('bar2h_list',bar2h_list)
    '''
    #return legend_names, values    
    #return return_df  
    return census_dict 

In [7]:
def build_state_dict(file_path):
    '''
    Read in a pathname for a CVS file
    
    Retrieve each record from a CSV file of state abbreviations
    
    Build dictionary that contains abbrevation, state name and state number used in census call
    
    '''
    file = open(file_path, newline='')
    reader = csv.DictReader(file)
    
    #build dictionary entry for each record read
    for row in reader:
        #print(row)
        state_code = row['st']
        state_name = row['name']
        state_number = row['code']
        state_dict[state_code] = [state_name,state_number]

In [8]:
def state_abbrev():
    '''
    Prompts for state abbreviation
    Returns state number used in census call
    
    '''
    
    not_validiated = True
    while not_validiated:
        input_state = input('Enter the state abbrevation: \n')
        input_state_u = input_state.upper()
        call_state = state_dict.get(input_state_u,0)
        if call_state == 0:
            print('Not valid input, please try again')
        else:
            #print('call_state',call_state)
            state_number = call_state[1]
            state_name = call_state[0]
            #print('state_number', state_number)
            not_validiated = False 
                        
    return(state_number, input_state_u )       

In [9]:
def graph_two_districts(l_names, x_names, l_bar1, l_bar2, y_pos, graph_title):
    '''
    creates side by side bar graphs
    l_names - legend
    x_names - names on x axis
    l_bar1 - list of values for first bar graph
    l_bar2 - list of values for second bar graph
    y_pos - list of length of bar graphs values
    graph_title - title of the graph
    '''
    #print('l_bar1',l_bar1)
    #print('l_bar2',l_bar2)
    #print('y_pos',y_pos)
    #print('x_names',x_names)
    
#graph total calls vs redacted calls
    width=0.3
    legend = l_names
    bar1 = plt.bar(y_pos, l_bar1, width=width)
    bar2 = plt.bar(y_pos+ width, l_bar2, width=width)
    bar3 = plt.bar(y_pos+ width, l_bar2, width=width)
    plt.grid(color='black', linestyle='--', linewidth=2, axis='y', alpha=0.7)
    plt.xticks(y_pos, x_names)
    plt.ylabel('Percentage- total maybe less 100%')
    #plt.title('Population makeup for the requested counties')
    plt.title(graph_title)
    #loc=0 is best position as determined by matplotlib
    #i think loc=6 is better
    plt.legend(legend,loc=0)
    #add text to top of bar
    for bar in bar1 + bar2 + bar3:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/3, height,
                 f'{height:,.0f}',
                 ha='center',
                 va='bottom')
    plt.show() 

In [10]:
def call_census(endpoint_ext):
    '''
    Call census api for American Community Survey (ACS) Year=2019 and Data Profiles tables
    Returns JSON format results
    '''
    races = 'DP05_0033E,DP05_0038E,DP05_0039E,DP05_0044E,DP05_0052E,DP05_0057E,DP05_0037E,DP05_0071E'
    root = 'https://api.census.gov/data/2019/acs/acs5/profile?get=NAME,'
    url = root + races + endpoint_ext
    response = requests.get(url)
    #print(url)
    #print(response)
    if (int(response.status_code)==200):
        #print('response',response.text)
        return response.text
    else:
        raise Exception("Non 200 status code") 

In [11]:
def get_population(state_num):
    '''
    builds variable for all counties within the selected state 
    Returns JSON format results
    '''
    #county_state = '&in=state:' + state_num
    state = '&for=state:' + state_num
    congressal_state = '&for=congressional%20district:*&in=state:' + state_num
    #raw_response = call_census(county_state)
    raw_response = call_census(congressal_state)
    raw_response_st = call_census(state)
    print('raw_response_st', type(raw_response_st),raw_response_st)
    #json_response = json.loads(raw_response)
    #print('json_response', json_response)
    
    return raw_response, raw_response_st

In [12]:
#main code
#create empty dictionary
state_dict = {}
#build dictionary of state abbrevations, names and numbers
build_state_dict('./data/state_table.txt')
#pp(state_dict)

#prompt for state abbrevation and return state number used in census call
state_number, state_cd = state_abbrev()
#print('state_number', state_number)

#call census API with state_number
district_data, state_data = get_population(state_number)
#print(census_data)

#legend_names, names, bar1_list, bar2_list, y_pos,bar1h_list, bar2h_list, namesh = build_census_dict(census_data, state_cd)
#census_df = build_census_dict(census_data, state_cd)
#build dictionary with district data
district_dict = build_census_dict(district_data, state_cd)

district_names = list(district_dict.keys())
district_names.sort()
    
district_values = list(district_dict.values())

#prompt for which districts to be selected
selected_districts = select_districts(district_names)
#print('selected_districts',selected_districts) 

#build dictionary with stare data
state_dict = build_census_dict_st(state_data, state_cd)

print('state_dict',state_dict) 
    
#build a list of lists based on the selected disticts    
census_lists = lists_from_dictionary_2(district_values,selected_districts)

#build dataframe from list of lists
census_df = build_df(census_lists)

#add percentages to dataframe
census2_df = percentages_df(census_df)

'''
legend_names.sort()

print('legend_names', type(legend_names), legend_names)
print('names', names)
print('bar1_list',bar1_list)
print('bar2_list',bar2_list)
print('y_pos', y_pos)
'''
#first graph break down of races in the two counties
graph_title = 'Population makeup for the requested counties'
#graph_two_districts(legend_names, names, bar1_list, bar2_list, y_pos,graph_title)

Enter the state abbrevation: 
pa
raw_response_st <class 'str'> [["NAME","DP05_0033E","DP05_0038E","DP05_0039E","DP05_0044E","DP05_0052E","DP05_0057E","DP05_0037E","DP05_0071E","state"],
["Pennsylvania","12791530","1430664","24691","436324","4198","275177","10300602","935216","42"]]
Select up to 5 of districts.  Enter zero to stop selecting
Valid districts numbers: 1 thru 18 (inclusive)
Select ditrict: 
1
Select ditrict: 
2
Select ditrict: 
3
Select ditrict: 
0
len_b 3
column_names ['state', 'district', 'total', 'black', 'native amer', 'asian', 'pacific is', 'other', 'white', 'hispanic']
row_values [['PA', 1, 709812, 29383, 1209, 42374, 145, 9542, 612387, 38390], ['PA', 2, 737498, 202618, 3011, 57256, 333, 90410, 358376, 188841], ['PA', 3, 722876, 411905, 2318, 42870, 426, 9939, 235443, 34226]]
  state  district   total   black  native amer  asian  pacific is  other  \
0    PA         1  709812   29383         1209  42374         145   9542   
1    PA         2  737498  202618         3